# **Welcome to Your First META-Powered Document Assistant**

Today, we’re going to build a simple but powerful AI tool:
a ducument Question-and-Answer system powered by Meta’s Llama AI models.

### **This tool will allow you to:**

Upload any document (PDF, DOCX, TXT, Markdown)

For example:

*   A company report
*   A curriculum
*   A policy document
*   A research paper


**Ask natural questions**

Like:


*   “What are the main goals in this document?”
*   “What does this policy say about attendance?”
*   “Summarise the key points in chapter one.”


🤖 And get **accurate** answers—based only on the **document(s)** itself/themselves

The AI does not guess or make things up.
It searches the document, finds the right information, and gives you a clean answer.





# **Before You Begin**

Please ensure you have the following:

1. Access to Google Colab

2. OpenRouter API Key - https://www.youtube.com/watch?v=-X9DVzzxpAA

3. Google Drive link to the document (s)

🤖 - "May the code be with you"

# Step 1 — Installing the Tools We Need

Before we can start building our AI system, we need to install a few essential tools. Think of this like installing apps on your phone before you can use them.

**What you'll need:**

1.   **LlamaIndex** — helps the AI read and understand documents
2.   **OpenRouter connector** — allows us to use Meta’s Llama AI model
3.   **HuggingFace Embeddings** — helps the AI understand text by turning words into numbers
4.  **Document readers** — so the notebook can open and read document (PDF, DOCX, TXT, Markdown)files
5.  **Fusion retriever tools** — advanced search tools that improve answer accuracy

This cell does not build the system yet — it only prepares the environment.
Once everything is installed, the rest of the notebook will work smoothly.

After you run this cell once, you don’t need to install again unless the runtime resets.

In [5]:
# Step 1 – Install all the tools our AI document assistant needs
#
# In this cell, we install the main libraries that make our RAG (Retrieval-Augmented Generation)
# system work. Think of this as installing the “apps” our notebook will use.
#
# Here is what each tool does:
#
# llama-index
#   - The main framework we use to read documents, break them into pieces, index them,
#     and search through them. It handles most of the heavy lifting for our RAG system.
#
# llama-index-llms-openrouter
#   - Allows us to connect to Meta’s Llama models through OpenRouter, so our AI can
#     understand questions and generate answers.
#
# llama-index-embeddings-huggingface
#   - Helps convert text into embeddings (numbers that the AI can understand).
#     This is what allows the system to find the right parts of the PDF when you ask a question.
#
# llama-index-readers-file
#   - Gives the notebook the ability to read PDF files and other documents from your system.
#
# llama-index-packs-fusion-retriever
#   - Provides the “query fusion” tool we use to improve answer accuracy. It takes your question,
#     rewrites it in different ways, searches the document multiple times, and combines the results.
#
# sentence-transformers
#   - A library that helps with semantic understanding. It is used for intelligent document chunking.
#
# nest-asyncio
#   - Allows certain parts of the system to run smoothly inside Google Colab by letting
#     asynchronous code work without errors.
#
# requests
#   - Helps us download the PDF directly from a Google Drive link.
#
# After running this cell, all the required tools will be installed and ready.
# You only need to run this once per session.
#

%pip install -q \
  llama-index \
  llama-index-llms-openrouter \
  llama-index-embeddings-huggingface \
  llama-index-readers-file \
  llama-index-packs-fusion-retriever \
  sentence-transformers \
  nest-asyncio \
  requests

print("✅ Installation complete")


✅ Installation complete


# Step 2 — Setting Up the AI Model

Now that all our tools have been installed, this next cell is where we connect our notebook to the actual AI model that will answer our questions.

In this step, we:

✔ 1. Enter our OpenRouter API key

✔ 2. Choose which Llama model we want to use

✔ 3. Set rules for how the AI should behave

✔ 4. Set up the embedding model

This is a tool that helps the AI “understand” text by turning it into numerical patterns.
It improves how well the system can find information inside the PDF.

**Instructions:**

Make sure you import all the required tools and libraries in the cell below—these are essential for building your RAG system.

**Hint:**



```
import os
from getpass import getpass
import nest_asyncio
nest_asyncio.apply()
```



In [6]:
# Step 2 – Connect to the AI model and set up how our system will think
#
# In this cell, we prepare the AI model that will answer our questions.
# Everything here is about giving the AI access, choosing the model we want,
# and making sure it behaves correctly.

# ENTER CODE HERE (IMPORTS)
import os
from getpass import getpass
import nest_asyncio
nest_asyncio.apply()


# Ask the user to enter their OpenRouter API key.
# This works like a password that allows us to use the Llama AI model.
os.environ["OPENROUTER_API_KEY"] = getpass("Enter your OpenRouter API key: ")

from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Here we choose which Llama model we want to use.
# - This model understands the questions we ask
# - It helps generate clear, readable answers
llm = OpenRouter(
    api_key=os.environ["OPENROUTER_API_KEY"],  # Connect using our key
    model="meta-llama/llama-3.3-70b-instruct:free",  # Choose the AI model
    max_tokens=512,  # Maximum size of the answer
    temperature=0.1, # Lower = more accurate and less creative
    timeout=60,      # Give the model time to respond
    system_prompt=(
        "You are a document-grounded RAG assistant.\n\n"
        "You can read and answer questions from the following document types:\n"
        "PDF, DOCX, TXT, and Markdown.\n\n"
        "Rules:\n"
        "- Use ONLY the provided document context.\n"
        "- Do NOT use external knowledge or assumptions.\n"
        "- If the answer is not present, say exactly: \"The document does not contain this information.\"\n"
        "- Never hallucinate, guess or make up facts..\n\n"
        "Citation enforcement:\n"
        "- Every factual statement MUST be supported by a citation.\n"
        "- Citations must include page numbers (for PDFs) or section/headings (for DOCX, TXT, Markdown).\n"
        "- Use the citation format: (Page X) or (Section: Heading Name).\n"
        "- If page or section information is missing from the context, state that citations are unavailable.\n\n"
        "Response format:\n"
        "- First, provide a clear answer in 2-5 lines.\n"
        "- Then provide a summary using bullet points (3-6 bullets).\n"
        "- Each point or section written MUST end with a citation.\n"
        "- Do not repeat information unnecessarily."
    ),
)

# This model helps the AI understand text in the PDF by turning words into numbers.
# This step helps the system search the document more effectively.
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# Finally, we tell the system to use the AI model and text-understanding model
# we just set up. This makes everything ready for the next steps.
Settings.llm = llm
Settings.embed_model = embed_model

print("✅ AI model and settings are ready to use")


Enter your OpenRouter API key: ··········


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ AI model and settings are ready to use


__Step 3 — Loading Documents From Google Drive (PDF, DOCX, TXT, Markdown, or Google Sheets)__

In this step, we provide our AI system with the documents it needs to learn from.
Instead of uploading files manually, we simply paste a Google Drive link, and the notebook automatically retrieves the documents for us.

The link can point to:

A single document (PDF, DOCX, TXT, or Markdown), or

A Google Sheet containing links to multiple documents

This cell:

✔ 1. Asks you to paste a Google Drive link

This can be a direct document link or a Google Sheet link.

✔ 2. Detects the type of link provided

The notebook determines whether the link points to:

__A document file, or__

__A Google Sheet containing document links__

✔ 3. Extracts the file ID(s)

The system identifies the correct Google Drive file IDs needed for downloading.

✔ 4. Downloads the document(s) automatically

If a single document link is provided, it is downloaded directly.

If a Google Sheet is provided, the notebook:

__Opens the sheet__

Extracts document links from the document_link column

Downloads each linked document

✔ 5. Saves all documents in a data folder

Downloaded files are saved as:

 source1.pdf

source2.docx


source3.md

These files become the core knowledge source for the AI system.

In [7]:
# ============================================================
# STEP 3 — GOOGLE DRIVE DOCUMENT & SHEET LOADER (FIXED)
# ============================================================

import os
import re
import requests
import pandas as pd
from io import BytesIO
from urllib.parse import urlparse

# ------------------------------------------------------------
# Setup
# ------------------------------------------------------------
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

SUPPORTED_EXTENSIONS = (".pdf", ".docx", ".txt", ".md")

# ------------------------------------------------------------
# Google Drive helpers
# ------------------------------------------------------------
def extract_file_id(url: str) -> str:
    match = re.search(r"/d/([A-Za-z0-9_-]+)", url) or \
            re.search(r"id=([A-Za-z0-9_-]+)", url)
    if not match:
        raise ValueError("❌ Could not extract Google Drive file ID.")
    return match.group(1)


def is_google_sheet(url: str) -> bool:
    return "spreadsheets/d" in url


def download_drive_file(file_id: str, save_path: str):
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    resp = requests.get(url)
    resp.raise_for_status()
    with open(save_path, "wb") as f:
        f.write(resp.content)


def load_google_sheet(file_id: str) -> pd.DataFrame:
    """
    Export Google Sheet as XLSX and load into pandas safely
    """
    url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx"
    resp = requests.get(url)
    resp.raise_for_status()
    return pd.read_excel(BytesIO(resp.content))

# ------------------------------------------------------------
# MAIN EXECUTION
# ------------------------------------------------------------
drive_link = input("📌 Paste Google Drive document or Google Sheet link: ").strip()
file_id = extract_file_id(drive_link)

download_count = 0

# ------------------------------------------------------------
# CASE 1 — GOOGLE SHEET (LINK LIST)
# ------------------------------------------------------------
if is_google_sheet(drive_link):
    print("📊 Google Sheet detected — extracting document links...")

    df = load_google_sheet(file_id)

    # Prefer explicit column name, fallback to second column
    if "document_link" in df.columns:
        links = df["document_link"].dropna().tolist()
    else:
        links = df.iloc[:, 1].dropna().tolist()  # second column

    for link in links:
        try:
            doc_id = extract_file_id(link)
        except ValueError:
            continue  # skip invalid links

        # Try to infer extension from URL path
        ext = os.path.splitext(urlparse(link).path)[1].lower()

        # If no extension in URL, assume PDF (Drive hides extensions)
        if ext not in SUPPORTED_EXTENSIONS:
            ext = ".pdf"

        download_count += 1
        save_path = os.path.join(DATA_DIR, f"source{download_count}{ext}")

        print(f"📥 Downloading {save_path}")
        download_drive_file(doc_id, save_path)

# ------------------------------------------------------------
# CASE 2 — SINGLE DOCUMENT
# ------------------------------------------------------------
else:
    ext = os.path.splitext(urlparse(drive_link).path)[1].lower()
    if ext not in SUPPORTED_EXTENSIONS:
        ext = ".pdf"  # default for Drive links

    save_path = os.path.join(DATA_DIR, f"source1{ext}")
    print(f"📥 Downloading {save_path}")
    download_drive_file(file_id, save_path)
    download_count = 1

print(f"\n✅ Successfully downloaded {download_count} document(s)")


📌 Paste Google Drive document or Google Sheet link: https://docs.google.com/spreadsheets/d/1Ho_diKCgVt4IZ2o0uRDEbY3Q5RKNpxk3fRnYIJzcNIw/edit?gid=0#gid=0
📊 Google Sheet detected — extracting document links...
📥 Downloading data/source1.pdf
📥 Downloading data/source2.pdf

✅ Successfully downloaded 2 document(s)


# **Step 4 — Preparing the Documents for the AI to Understand**

Now that we’ve downloaded our documents from Google Drive, the next step is **to help the AI make sense of their content.**

AI models cannot read full documents the way humans do. Instead, they work best when text is broken into small, meaningful sections. This step is critical because it directly affects how accurately the AI will retrieve information and answer questions later.

**In this cell, we:**

**✔ 1. Load all downloaded documents into the notebook**

The system reads every supported file (PDF, DOCX, TXT, and Markdown) from the data/ folder so they are ready for processing.

**✔ 2. Break the documents into “semantic chunks”**

Rather than cutting text at fixed lengths, we split documents based on meaning.
This is similar to dividing a book into logical paragraphs or sections, allowing the AI to understand the content more naturally.

**✔ 3. Add simple metadata to each chunk**

Each chunk is labeled with its source file and type.
This makes it easy to trace answers back to the original document and enables citations later.

**✔ 4. Prepare the chunks for intelligent searching**

These semantic chunks are the building blocks the AI will later search through when responding to user questions.

**In simple terms:**

**Step 4 teaches the AI how to read and understand multiple** documents in a structured way, so it can accurately search through them and return relevant answers.

__The output of this step is a collection of clean, meaningful text chunks that are ready for embedding, indexing, and question answering in the next stages of the system.__

In [8]:
# ============================================================
# STEP 4 —  Break/ PREPARE DOCUMENTS INTO MEANINGFUL PIECES THE AI CAN UNDERSTAND
#
# In this step, we take the documents downloaded in Step 3
# (PDF, DOCX, TXT, Markdown) and break them into small,
# meaningful pieces that the AI can read it properly, understand and search.


# Cell 4 – Break the PDF into meaningful pieces the AI can understand
#
# In this step, we take the PDF we downloaded and prepare it so the AI can
# read it properly. The AI cannot understand one big block of text, so we
# break the document into smaller, meaningful sections (called “chunks”).

# In other words, we use semantic chunking to split documents based on meaning
# rather than random size.
#
# These chunks act like paragraphs or mini-sections that the AI can search
# through when answering your questions.

# ============================================================

import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ------------------------------------------------------------
# Load all documents downloaded in Step 3
# ------------------------------------------------------------
# The 'data/' folder contains files like:
# source1.pdf, source2.docx, source3.txt, source4.md, etc.
#
# SimpleDirectoryReader automatically detects, loads
# and then reads the  supported file types into the system.
DATA_DIR = "data"

documents = SimpleDirectoryReader(DATA_DIR).load_data()
print(f"📄 Loaded {len(documents)} document(s) from '{DATA_DIR}' folder.")

# ------------------------------------------------------------
# Create the embedding model used for semantic understanding
# ------------------------------------------------------------
# We use an embedding model to help the AI understand the meaning of the text.
# This model helps the system decide where natural breaks (sections) should be

semantic_embed = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# ------------------------------------------------------------
# Create the semantic chunker (a tool that will split the PDF into smart, meaningful pieces.)
# ------------------------------------------------------------
# Instead of splitting text randomly, this parser:
# - Looks at semantic meaning (it looks at the meaning of each sentence)
# - Keeps related sentences together
# - so the chunks feel more natural (like real sections).
# - This Produces high-quality chunks for accurate retrieval

parser = SemanticSplitterNodeParser(
    buffer_size=3,                     # Helps keep small related groups together
    breakpoint_percentile_threshold=95, # Controls split sensitivity (how sensitive the splitting is)
    embed_model=semantic_embed # Uses the meaning of the text for splitting
)

# ------------------------------------------------------------
# Break documents into semantic chunks
# Use the parser to turn the document into these small, meaningful chunks.
# ------------------------------------------------------------
nodes = parser.get_nodes_from_documents(documents)

# ------------------------------------------------------------
# Add metadata to each chunk
# ------------------------------------------------------------
# Add simple labels (metadata ) to each chunk so we know where it came from.
# This is helpful when the AI searches for answers.

# Metadata helps with:
# - Tracing answers back to source files
# - Enabling citations later
for node in nodes:
    node.metadata["source"] = node.metadata.get("file_name", "unknown")
    node.metadata["chunk_type"] = "semantic"

print(f"🔍 Created {len(nodes)} high-quality semantic chunks.")

# The variable `nodes` now contains clean, meaningful chunks
# ready for embedding, indexing, and retrieval in later steps.


📄 Loaded 107 document(s) from 'data' folder.
🔍 Created 204 high-quality semantic chunks.


# **Step 5 — Building the AI Search Engine**

Now that our documents have been broken into small, meaningful pieces, we are ready to build the search engine part of our AI system.
This is one of the most important steps in the entire pipeline.

At this stage, the AI already understands the documents at a structural level. What we build here determines how well it can find the right information when you ask a question.

**In this cell, we build a system that helps the AI:**

✔ Understand your question

✔ Rewrite your question in different ways

✔ Search through the document chunks multiple times

✔ Compare and combine the best results

✔ Return the most accurate and reliable answer possible

**What is Query Fusion?**

This step uses a technique called Query Fusion, inspired by __Meta’s Llama Cookbook.__ It makes the AI much smarter and more reliable—especially when questions are complicated or phrased in different ways.

__Instead of searching the documents only once, Query Fusion:__

- Rewrites your question into several alternative forms

- Runs multiple searches across the document chunks

- Fuses the strongest results into a single, high-quality response


_Think of it like having multiple researchers independently search the documents, then combining their findings into one clear answer._

**In simple terms:**

Step 5 builds the “brain” of our AI search system — the part that allows the AI to find relevant information even when questions are complex, indirect, or phrased in different ways.

Instructions:

Set up the advanced retriever that will improve your RAG system’s accuracy by rewriting queries, performing multiple searches, and combining the results using the semantic chunks created in Step 4.

In [9]:
# ============================================================
# STEP 5 — BUILD THE AI SEARCH ENGINE (QUERY FUSION)
#
# In this step, we take the semantic chunks created in Step 4
# and build an advanced retrieval system.
#
# This system improves accuracy by:
#   - Rewriting the user’s question multiple times
#   - Searching the document chunks repeatedly
#   - Combining the strongest results into one final answer
#
# This approach is known as Query Fusion.
# ============================================================

from llama_index.core.llama_pack import download_llama_pack

# ------------------------------------------------------------
# Download the Query Fusion retriever pack
# ------------------------------------------------------------
# This is downloaded only once. On future runs, it will be
# loaded from the local directory.
QueryRewritingRetrieverPack = download_llama_pack(
    "QueryRewritingRetrieverPack",
    "./query_rewriting_pack",
)

# ------------------------------------------------------------
# Build the advanced retriever using Step 4 chunks
# ------------------------------------------------------------
# We pass in the semantic chunks ("nodes") generated earlier.
# These chunks already contain meaning-aware splits and metadata.
query_rewriting_pack = QueryRewritingRetrieverPack(
    nodes=nodes,                   # Semantic chunks from Step 4
    chunk_size=256,                # Logical size for retrieval
    vector_similarity_top_k=8,     # Initial similarity search depth
    fusion_similarity_top_k=8,     # Final fused result depth
    num_queries=6                  # Number of rewritten queries
)

print("🚀 Advanced Query Fusion RAG Engine is ready!")


🚀 Advanced Query Fusion RAG Engine is ready!


In [ ]:
# Cell 5 – Build the smart search system (Query Fusion)
#
# Now that we have broken the PDF into small, meaningful pieces (chunks),
# this cell builds the “search engine” that will help the AI find the
# right information when you ask a question.
#
# This uses a method called **Query Fusion**, which comes from Meta’s Llama Cookbook.
# Query Fusion improves accuracy by:
#   - Rewriting your question in different ways
#   - Searching the document several times
#   - Combining the best results into one strong answer
#
# Think of it like asking several people the same question and then blending
# their answers to get the most reliable one.

from llama_index.core.llama_pack import download_llama_pack

# Download the special Query Fusion tool.
# This only downloads the first time — after that it loads from your system.
QueryRewritingRetrieverPack = download_llama_pack(
    "QueryRewritingRetrieverPack",
    "./query_rewriting_pack",
)

# ENTER CODE HERE (ADVANCED RETRIEVER CREATION)
# Build the advanced search system using our document chunks.
# Here we set how many searches it should perform and how much information to gather.
query_rewriting_pack = QueryRewritingRetrieverPack(
    nodes,                        # The chunks created in Step 4
    chunk_size=256,               # The size of each text chunk to consider
    vector_similarity_top_k=8,    # How many top matches to look at in the first layer of search
    fusion_similarity_top_k=8,    # How many matches to merge for the final answer
    num_queries=6                # Rewrites your question 6 different ways for a stronger search
)

print("🚀 Advanced Query Fusion RAG Engine Ready!")

🚀 Advanced Query Fusion RAG Engine Ready!


Step 6 — Asking Questions and Getting Answers

This is the final step — and the part you’ll interact with the most.

Now that our AI search engine has been built, this cell creates a simple  chat-like interactive loop where you can ask questions and receive answers grounded entirely in your documents.

At this point, the system has:

Read and understood all documents

Broken them into meaningful chunks

Built a smart search engine using Query Fusion

In this step, you can:

✔ Ask any question about the documents

✔ Watch the AI search through multiple chunks

✔ Receive clear, accurate answers based only on the documents

✔ Ask as many questions as you want

✔ Type “end” to stop the session


__What happens behind the scenes?__

When you ask a question, the AI:

1. Reads your question

2. Rewrites it in multiple ways (Query Fusion)

3. Searches the document chunks several times

4. Compares and merges the best results

5. Generates a grounded, reliable answer

This effectively turns your document collection into a smart assistant that can explain, summarize, or locate information instantly.

__In simple terms:__

_Step 6 is where you finally talk to the AI.
You ask questions, the AI searches intelligently, and answers come directly from your documents — nothing else._

Instructions:

Create an interactive loop that allows users to ask questions and receive real-time answers from the Query Fusion RAG system built in Step 5.

In [10]:
# ============================================================
# STEP 6 — INTERACTIVE QUESTION & ANSWER LOOP
#
# This final cell lets you interact with your RAG system.
# This cell creates a simple question-and-answer loop.
# You can ask questions about the documents, and the AI will:
#   - Rewrite your question (Query Fusion)
#   - Search the semantic chunks
#   - Combine the best results
#   - Return a clear, document-grounded answer
# The loop continues until you type "end", which stops the system.
# ============================================================

def safe_rag_run(question, retries=3):
    """
    This function safely runs the AI search (runs a query through the Query Fusion RAG system.)
    If something goes wrong (like a slow internet response),
    It will try again up to 3 times before giving up.

    """
    for attempt in range(retries):
        try:
          # Ask the AI to answer the question based on the document(s) content.
            response = query_rewriting_pack.run(question)

            # If the AI returns nothing, treat it as an error.
            if response is None or str(response).strip() == "":
                raise ValueError("Empty response returned by RAG system.")

            return response

        except Exception as e:
            # If something goes wrong, show the error and retry.
            print(f"⚠️ Error: {e}")
            print(f"🔁 Retrying ({attempt + 1}/{retries})...")

    # If all retries fail, return a message saying we couldn’t get an answer.
    return "❌ Unable to generate a valid answer after multiple attempts."


# ------------------------------------------------------------
# Interactive loop
# ------------------------------------------------------------

print("\n🤖 RAG Interactive Mode")
print("Ask any question about the documents.")
print("Type 'end' to exit.\n")
# This loop keeps running until the user types "end".
while True:
    # Ask the user to type a question.
    user_question = input("🟦 Enter your question: ").strip()

    # If the user types "end", stop the loop and exit.
    if user_question.lower() == "end":
        print("\n👋 Session ended.")
        break

    print("\n🔍 Searching documents...\n")

    # Run the question through our safe search function.
    answer = safe_rag_run(user_question)

    # Display the question and the AI’s answer.
    print("\n──────────────────────────────────────────────")
    print("❓ QUESTION:")
    print(user_question)
    print("\n🧠 ANSWER:")
    print(answer)
    print("──────────────────────────────────────────────\n")



🤖 RAG Interactive Mode
Ask any question about the documents.
Type 'end' to exit.

🟦 Enter your question: summarise the documents

🔍 Searching documents...

Generated queries:
How to summarize long documents effectively
Document summarization techniques and tools
Best practices for summarizing complex documents
Automated document summarization software and algorithms
Methods for summarizing large volumes of documents quickly and accurately

──────────────────────────────────────────────
❓ QUESTION:
summarise the documents

🧠 ANSWER:
The documents appear to be related to Artificial Intelligence (AI) strategies and plans from two different sources: Mauritius and Australia. The documents discuss various aspects of AI, including its applications, benefits, challenges, and implementation (Page 25). They also outline national AI plans, including goals, actions, and strategies for capturing opportunities, spreading benefits, and keeping people safe (Page 4). The documents highlight the import